# X ray Image Classification (Model 1)

## Table of Content
<ul>
    <li><a href="#dr">Data Reading.</a></li>
    <li><a href="#vgg">Loading VGG16 Model.</a></li>
    <li><a href="#dp">Data Preprocessing.</a></li>
    <li><a href="#fdl">Freezed Model Training.</a></li>
    <li><a href="#ufdl">Unfreezed Model Training.</a></li>
    <li><a href="#e">Evaluation and Saving.</a></li>
</ul>

<a id="dr"></a>
## Data Reading

In [1]:
! mkdir data
! unzip /content/train.zip -d data

Archive:  /content/train.zip
   creating: data/train/
   creating: data/train/COVID PATIENTS/
  inflating: data/train/COVID PATIENTS/covid_0.jpg  
  inflating: data/train/COVID PATIENTS/covid_1.jpg  
  inflating: data/train/COVID PATIENTS/covid_10.jpg  
  inflating: data/train/COVID PATIENTS/covid_100.jpg  
  inflating: data/train/COVID PATIENTS/covid_101.jpg  
  inflating: data/train/COVID PATIENTS/covid_102.jpg  
  inflating: data/train/COVID PATIENTS/covid_103.jpg  
  inflating: data/train/COVID PATIENTS/covid_104.jpg  
  inflating: data/train/COVID PATIENTS/covid_105.jpg  
  inflating: data/train/COVID PATIENTS/covid_106.jpg  
  inflating: data/train/COVID PATIENTS/covid_107.jpg  
  inflating: data/train/COVID PATIENTS/covid_108.jpg  
  inflating: data/train/COVID PATIENTS/covid_109.jpg  
  inflating: data/train/COVID PATIENTS/covid_11.jpg  
  inflating: data/train/COVID PATIENTS/covid_110.jpg  
  inflating: data/train/COVID PATIENTS/covid_111.jpg  
  inflating: data/train/COVID PA

<a id="vgg"></a>

## Loading VGG16 Model.

In [2]:
from tensorflow import keras

base_model = keras.applications.VGG16(
    weights="imagenet",
    input_shape=(224, 224, 3),
    include_top=False)

58900480/58889256 [==============================] - 1s 0us/step


In [3]:
# Freeze base model
base_model.trainable = False

In [4]:
inputs = keras.Input(shape=(224, 224, 3))

x = base_model(inputs, training=False)
x = keras.layers.GlobalAveragePooling2D()(x)

# A Dense classifier with a single unit (binary classification)
outputs = keras.layers.Dense(2, activation = 'softmax')(x)
model = keras.Model(inputs, outputs)

In [5]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 vgg16 (Functional)          (None, 7, 7, 512)         14714688  
                                                                 
 global_average_pooling2d (G  (None, 512)              0         
 lobalAveragePooling2D)                                          
                                                                 
 dense (Dense)               (None, 2)                 1026      
                                                                 
Total params: 14,715,714
Trainable params: 1,026
Non-trainable params: 14,714,688
_________________________________________________________________


In [6]:
model.compile(loss='categorical_crossentropy',
                  metrics=['accuracy'])

<a id="dp"></a>
## Data Pre-processing

In [7]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

transformation_ratio = 0.05
# Create a data generator

image_generator = ImageDataGenerator(
                              rescale=1. / 255,
                             validation_split = 0.2,
                             rotation_range=transformation_ratio,
                             shear_range=transformation_ratio,
                             zoom_range=transformation_ratio,
                             cval=transformation_ratio,
                             horizontal_flip=True,
                             vertical_flip=True)

In [8]:
# load and iterate training dataset
train = image_generator.flow_from_directory(
    "/content/data/train",
    target_size=(224, 224),
    color_mode="rgb",
    class_mode="categorical",
    subset='training'
)

valid = image_generator.flow_from_directory(
    "/content/data/train",
    target_size=(224, 224),
    color_mode="rgb",
    class_mode="categorical",
    subset='validation'
)

Found 260 images belonging to 2 classes.
Found 64 images belonging to 2 classes.


<a id="fdl"></a>
## Freezed Model Training

In [9]:
model.fit(train,
          validation_data=valid,
          steps_per_epoch=train.samples/train.batch_size,
          validation_steps=valid.samples/valid.batch_size,
          epochs=25)

Epoch 1/25
8/8 [==============================] - 172s 22s/step - loss: 0.7153 - accuracy: 0.4962 - val_loss: 0.6947 - val_accuracy: 0.5781
Epoch 2/25
8/8 [==============================] - 167s 21s/step - loss: 0.6843 - accuracy: 0.5962 - val_loss: 0.6861 - val_accuracy: 0.5156
Epoch 3/25
8/8 [==============================] - 168s 21s/step - loss: 0.6797 - accuracy: 0.6077 - val_loss: 0.6809 - val_accuracy: 0.5469
Epoch 4/25
8/8 [==============================] - 167s 21s/step - loss: 0.6677 - accuracy: 0.6115 - val_loss: 0.6638 - val_accuracy: 0.6250
Epoch 5/25
8/8 [==============================] - 165s 21s/step - loss: 0.6540 - accuracy: 0.7154 - val_loss: 0.6566 - val_accuracy: 0.7500
Epoch 6/25
8/8 [==============================] - 165s 21s/step - loss: 0.6427 - accuracy: 0.7538 - val_loss: 0.6499 - val_accuracy: 0.7031
Epoch 7/25
8/8 [==============================] - 166s 21s/step - loss: 0.6312 - accuracy: 0.7231 - val_loss: 0.6378 - val_accuracy: 0.6875
Epoch 8/25
8/8 [====

In [10]:
# Unfreeze the base model
base_model.trainable = True

# Compile the model with a low learning rate
model.compile(optimizer=keras.optimizers.RMSprop(learning_rate = .00001),
              loss = keras.losses.BinaryCrossentropy(from_logits=True) ,
              metrics = [keras.metrics.BinaryAccuracy()])

<a id="ufdl"></a>
## Unfreezed Model Training

In [11]:
model.fit(train,
          validation_data=valid,
          steps_per_epoch=train.samples/train.batch_size,
          validation_steps=valid.samples/valid.batch_size,
          epochs=10)

Epoch 1/10


/usr/local/lib/python3.7/dist-packages/tensorflow/python/util/dispatch.py:1082: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


8/8 [==============================] - 519s 64s/step - loss: 0.5998 - binary_accuracy: 0.6923 - val_loss: 0.5755 - val_binary_accuracy: 0.6875
Epoch 2/10
8/8 [==============================] - 517s 64s/step - loss: 0.4862 - binary_accuracy: 0.7962 - val_loss: 0.5386 - val_binary_accuracy: 0.6406
Epoch 3/10
8/8 [==============================] - 516s 71s/step - loss: 0.4231 - binary_accuracy: 0.8038 - val_loss: 0.4380 - val_binary_accuracy: 0.7969
Epoch 4/10
8/8 [==============================] - 519s 64s/step - loss: 0.3896 - binary_accuracy: 0.8423 - val_loss: 0.3818 - val_binary_accuracy: 0.8438
Epoch 5/10
8/8 [==============================] - 522s 72s/step - loss: 0.3483 - binary_accuracy: 0.8654 - val_loss: 0.3584 - val_binary_accuracy: 0.8281
Epoch 6/10
8/8 [==============================] - 524s 65s/step - loss: 0.2925 - binary_accuracy: 0.8846 - val_loss: 0.6788 - val_binary_accuracy: 0.6406
Epoch 7/10
8/8 [==============================] - 513s 64s/step - loss: 0.3651 - binary

<a id="e"></a>
## Evaluation and Testing

In [12]:
model.evaluate(valid, steps=valid.samples/valid.batch_size)

2/2 [==============================] - 33s 16s/step - loss: 0.2709 - binary_accuracy: 0.8906


[0.27089232206344604, 0.890625]

In [13]:
model.save("model_v1")

INFO:tensorflow:Assets written to: model_v1/assets
